# Habrahabr news scrapper

Magic функция, которая будет одновременно выполняет код ячейки и экспортирует его в файл, так как встроенная функция %%writefile такой функциональности не поддерживает.

In [1]:
from IPython.core.magic import register_cell_magic


@register_cell_magic
def write_and_run(line, cell):
    argz = line.split()
    file = argz[-1]
    mode = "w"
    if len(argz) == 2 and argz[0] == "-a":
        mode = "a"
    with open(file, mode) as f:
        f.write(cell + "\n\n")
    get_ipython().run_cell(cell)

Секция импорта

In [2]:
%%write_and_run scrapper.py
import locale
import unicodedata

import requests
from fake_useragent import UserAgent
from lxml import html

locale.setlocale(locale.LC_TIME, "ru_RU.UTF-8")

'ru_RU.UTF-8'

Скачиваем и парсим данные

In [3]:
%%write_and_run -a scrapper.py
# fake agent
user_agent = UserAgent().chrome

In [9]:
user_agent

'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.2117.157 Safari/537.36'

In [4]:
%%write_and_run -a scrapper.py
# init requests session
session = requests.Session()
session.headers.update(
    {"User-Agent": user_agent,}
)

In [5]:
%%write_and_run -a scrapper.py
# download page
url = "https://habr.com/ru/news/"
page = session.get(url)
content = page.text
parsed = html.fromstring(content)

In [6]:
%%write_and_run -a scrapper.py
# parse page
data = []
headers = parsed.xpath('//h2[@class="post__title"]')
for header in headers:
    _id = header.xpath('./a[@class="post__title_link"]/@href')[0]
    text = header.xpath('./a[@class="post__title_link"]/text()')[0]
    text = unicodedata.normalize("NFKD", text)
    data.append({"_id": _id, "text": text})

Сохраняем данные в S3

In [7]:
%%write_and_run -a scrapper.py
# make connection to s3
import boto3

session = boto3.session.Session()
s3 = session.client(service_name="s3", endpoint_url="https://storage.yandexcloud.net")

In [8]:
%%write_and_run -a scrapper.py
# overwrite data
storage_data = ".\n".join([x["text"] for x in data])
s3.put_object(Bucket="scrapper", Key="habr/news", Body=storage_data)

{'ResponseMetadata': {'RequestId': '67176b2d5036b53b',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx',
   'date': 'Sun, 26 Jul 2020 17:55:32 GMT',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'keep-alive': 'timeout=60',
   'etag': '"62196c973fa64c09848c2de86ddfdd26"',
   'x-amz-request-id': '67176b2d5036b53b',
   'x-amz-version-id': 'null',
   'x-yc-s3-cloud-id': 'b1g4l6iqet969vscav0l',
   'x-yc-s3-folder-id': 'b1glgqadrs39qkbojcqo'},
  'RetryAttempts': 0},
 'ETag': '"62196c973fa64c09848c2de86ddfdd26"',
 'VersionId': 'null'}

In [ ]:
# Получить список объектов в бакете
for key in s3.list_objects(Bucket="scrapper")["Contents"]:
    print(key["Key"])

In [ ]:
# Получить объект
get_object_response = s3.get_object(Bucket="scrapper", Key="habr/news")
response_body = get_object_response["Body"].read()

In [ ]:
# Парсинг тела
response_parsed = response_body.decode("utf-8").split('\n')
response_parsed

In [ ]:
# Ограничение по размеру
size = 0
text_body = str()
for header in response_parsed:
    size += len(header) + len(' ')
    if size > 1024:
        break
    text_body += header + ' '
text_body

Сохраняем данные в MongoDB

In [ ]:
%%write_and_run -a scrapper.py
# make connection to mongo
from pymongo import MongoClient

mongo = MongoClient("localhost", 27017)
db = mongo.habr
collection = db.news

In [ ]:
%%write_and_run -a scrapper.py
# update database
for item in data:
    if (
        collection.update_one(
            {"_id": item["_id"]}, {"$set": item}, upsert=True
        ).matched_count
        > 0
    ):
        break

Полезные операции с монгой, чтобы каждый раз не набирать в консоли

In [ ]:
# Создание контейнера
!docker run -d -p 27017:27017 --name mongodb mongo 

In [ ]:
# Запуск контейнера
!docker start mongodb

In [ ]:
# Дамп базы 'habr' в рабочую директорию
!docker exec -it mongodb mongodump --out=/backup/ --db=habr
!docker exec -it mongodb tar czf dump.mongo.tgz /backup
!docker cp mongodb:/dump.mongo.tgz dump.mongo.tgz
!docker exec -it mongodb rm -rf /backup /dump.mongo.tgz

In [ ]:
# Восстановление данных из дампа
!docker cp dump.mongo.tgz mongodb:/dump.mongo.tgz
!docker exec -it mongodb tar xzf dump.mongo.tgz
!docker exec -it mongodb mongorestore /backup
!docker exec -it mongodb rm -rf /backup /dump.mongo.tgz

In [ ]:
# Остановка контейнера
!docker stop mongodb